In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.shape

(768, 9)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
df['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

## The Data is imbalanced we have to handle it.
1. Resample the imbalanced data
2. Use Precision recall and F1_score instead of accuracy score.

### Resampling the dataset

In [9]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Harshit\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [12]:
X = df.drop('Outcome', axis = 1)
y = df['Outcome']

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [25]:
## Standardize the X_train, X_test
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
y_train.value_counts()

Outcome
0    401
1    213
Name: count, dtype: int64

In [29]:
## We have to resample the trainging data so that the model genelarize well.
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 42)

In [30]:
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [31]:
y_train_resampled.shape

(802,)

## Model Creation

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import classification_report, accuracy_score

In [40]:
models = {
    "logistic_regression" : LogisticRegression(),
    "svm" : SVC(),
    "Naive_bayes" : GaussianNB(),
    "KNN" : KNeighborsClassifier(),
    "DecisionTree" : DecisionTreeClassifier(),
    "RandomForest" : RandomForestClassifier(),
    "Adaboost" : AdaBoostClassifier()
}
models

{'logistic_regression': LogisticRegression(),
 'svm': SVC(),
 'Naive_bayes': GaussianNB(),
 'KNN': KNeighborsClassifier(),
 'DecisionTree': DecisionTreeClassifier(),
 'RandomForest': RandomForestClassifier(),
 'Adaboost': AdaBoostClassifier()}

In [50]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test)

    print(f"{list(models.keys())[i]} : ")
    print(classification_report(y_test, y_pred))
    print(accuracy_score(y_test, y_pred)*100)
    print()

logistic_regression : 
              precision    recall  f1-score   support

           0       0.82      0.70      0.75        99
           1       0.57      0.73      0.64        55

    accuracy                           0.71       154
   macro avg       0.70      0.71      0.70       154
weighted avg       0.73      0.71      0.71       154

70.77922077922078

svm : 
              precision    recall  f1-score   support

           0       0.80      0.72      0.76        99
           1       0.57      0.67      0.62        55

    accuracy                           0.70       154
   macro avg       0.68      0.69      0.69       154
weighted avg       0.72      0.70      0.71       154

70.12987012987013

Naive_bayes : 
              precision    recall  f1-score   support

           0       0.83      0.72      0.77        99
           1       0.59      0.73      0.65        55

    accuracy                           0.72       154
   macro avg       0.71      0.72      0.71  

## RandomForest gives us the best results of both f1-score for minority class and accuracy_score.

### Applying hyperparameter tunning on it

In [52]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=50,
    cv=5,
    scoring='f1',  # Focus on minority class performance
    n_jobs=-1,
    verbose=1,
    random_state=42
)

random_search.fit(X_train_resampled, y_train_resampled)
best_model = random_search.best_estimator_


Fitting 5 folds for each of 50 candidates, totalling 250 fits


In [63]:
from sklearn.metrics import classification_report, f1_score

y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("f1-score : ", f1_score(y_test, y_pred) * 100)

              precision    recall  f1-score   support

           0       0.84      0.76      0.80        99
           1       0.63      0.75      0.68        55

    accuracy                           0.75       154
   macro avg       0.74      0.75      0.74       154
weighted avg       0.77      0.75      0.76       154

f1-score :  68.33333333333333


In [64]:
## The model {best_model} gives the highest value of minority class in F1-score thus this model can be use in production.

In [65]:
## prediction for new data
input_data = (1,198,12,95,10,83.6,40.627,90)
inp_arr = np.asarray(input_data)
reshape_data = inp_arr.reshape(1,-1)
reshape_data = scaler.transform(reshape_data)
prediction = best_model.predict(reshape_data)
print(prediction)

[0]


C:\Users\Harshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
